In [1]:
from cell_env import CellEnv
# Use sb3 env checker:
from stable_baselines3.common.env_checker import check_env
from ddqn import DoubleDQN

from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.wrappers import TimeLimit
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback


In [2]:
alpha=0.7
env_args = {
    "max_timesteps": 10000,
    "alpha_mem": alpha,
    "dt": 0.01,
    "frame_stack": 5,
}


In [3]:
env = CellEnv(**env_args)
check_env(env)
# use the monitor wrapper to log the results:
env = Monitor(env)
eval_env = CellEnv(**env_args)
eval_env = Monitor(eval_env)

In [4]:

eval_callback = EvalCallback(eval_env, best_model_save_path=f'./rl-models-sde_{alpha}/',
                             n_eval_episodes=1,
                             log_path='./rl-logs/', eval_freq=10_000,
                             deterministic=True, render=False,
                             )

# TODO Convert terminations to truncations; use tau<1

model = DoubleDQN("MlpPolicy", env,
                    verbose=4, tensorboard_log="./rl-logs/",
                    exploration_fraction=0.05,
                    exploration_final_eps=0.0,
                    target_update_interval=500,
                    buffer_size=100_000,
                    gradient_steps=-1,
                    train_freq=(1, "episode"),
                    learning_starts=10000,
                    learning_rate=0.00036,
                    batch_size=32,
                    gamma=0.995,
)
model.learn(total_timesteps=1_000_000, tb_log_name="dqn",
            callback=eval_callback)


Using cuda device
Wrapping the env in a DummyVecEnv.
Logging to ./rl-logs/dqn_76
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.75     |
|    ep_rew_mean      | -0.77    |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2767     |
|    time_elapsed     | 0        |
|    total_timesteps  | 15       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.75     |
|    ep_rew_mean      | -0.778   |
|    exploration_rate | 1        |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2620     |
|    time_elapsed     | 0        |
|    total_timesteps  | 22       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.33     |
|    ep_rew_mean      | -0.813   |
|    expl